In [1]:
import oqs
# Use the Kyber512 KEM algorithm
algorithm = 'Kyber512'
# --------- 1. Generate Keypair ---------
with oqs.KEM(algorithm) as server:
    print(f"\nAlgorithm: {algorithm}")
    # Server generates key pair (public, secret)
    public_key = server.generate_keypair()
    secret_key = server.export_secret_key()
    print(f"\n[1] Key Generation:")
    print(f"Public Key (len={len(public_key)}): {public_key.hex()}")
    print(f"Secret Key (len={len(secret_key)}): {secret_key.hex()}")

    # --------- 2. Client Encapsulation ---------
    with oqs.KeyEncapsulation(algorithm) as client:
        # Client uses server's public key to encapsulate a shared secret
        ciphertext, shared_secret_enc = client.encap_secret(public_key)
        print(f"\n[2] Encapsulation:")
        print(f"Ciphertext (len={len(ciphertext)}): {ciphertext.hex()}")
        print(f"Client's Shared Secret: {shared_secret_enc.hex()}")

    # --------- 3. Server Decapsulation ---------
    shared_secret_dec = server.decap_secret(ciphertext)
    print(f"\n[3] Decapsulation:")
    print(f"Server's Shared Secret: {shared_secret_dec.hex()}")

    # --------- 4. Derive symmetric key & Encrypt Plaintext ---------
    # For demonstration, we’ll encrypt plaintext with AES-GCM using shared secret
    from Crypto.Cipher import AES
    from Crypto.Random import get_random_bytes
    # Example plaintext
    plaintext = b'Post-Quantum Test'
    print(f"\n[4] Plaintext: {plaintext}")
    # Use first 16 bytes of shared secret for AES-128 key
    key = shared_secret_enc[:16]
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext_aes, tag = cipher.encrypt_and_digest(plaintext)
    print(f"Symmetric Key (AES-128): {key.hex()}")
    print(f"AES-GCM Nonce: {cipher.nonce.hex()}")
    print(f"AES-GCM Ciphertext: {ciphertext_aes.hex()}")
    print(f"AES-GCM Tag: {tag.hex()}")

    # --------- 5. Decrypt the AES ciphertext ---------
    cipher_dec = AES.new(key, AES.MODE_GCM, nonce=cipher.nonce)
    decrypted = cipher_dec.decrypt_and_verify(ciphertext_aes, tag)
    print(f"\n[5] Decrypted Plaintext: {decrypted}")


AttributeError: module 'oqs' has no attribute 'KEM'